# Validação de Modelos

A influência da aleatoriedade na validação do modelo

In [84]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [85]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [86]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


In [87]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


A decisão está dependendo da aleatoriedade. Se trocarmoso seed troca o valor. 

Como rodar o treino e teste uma única vez, e termos um intervalo de acurária

In [88]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 76.84%


Usando e avaliando com o cross validade

In [89]:
#Cv quantos pedaços
from sklearn.model_selection import cross_validate

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3)
results

{'fit_time': array([0.04691124, 0.00951624, 0.00698066]),
 'score_time': array([0.00397563, 0.00199533, 0.00099564]),
 'test_score': array([0.75704859, 0.7629763 , 0.75337534])}

In [90]:
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)

In [91]:
results['test_score']

array([0.75704859, 0.7629763 , 0.75337534])

In [92]:
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy %.2f %.2f" % (media - 2 * desvio_padrao, media + 2 * desvio_padrao))

Accuracy 0.75 0.77


In [93]:
# para dar o intervalo (media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100)
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy [74.99, 76.57]


In [94]:
# mudando cv muda o intervalo
# mudando o seed não muda o intervalo

from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.24, 77.32]


In [95]:
# recomendado usar 5
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


kfold com aleatorização

In [96]:
#Aleatoriedade no cross validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.24, 77.32]


In [97]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio: %.2f" % (media * 100))
    print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [98]:
from sklearn.model_selection import KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [74.37, 77.19]


In [99]:
# shuffle - Aleatório 
#Definindo uma variavel ‘ cv = KFold(n_splits = 10, shuffle = True) ‘ para embaralhar 10 elementos.


SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth = 2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.76
Accuracy intervalo: [73.26, 78.26]


Estratificação

In [100]:
#Simular situação horrível de azar
#Pode ser uma situação de "azar" ou uma proporção de exemplos desbalanceado entre as classes.

dados.sort_values("vendido", ascending=True)

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
...,...,...,...,...
5491,71910.43,1,9,25778.40812
1873,30456.53,1,6,15468.97608
1874,69342.41,1,11,16909.33538
5499,70520.39,1,16,19622.68262


In [101]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [102]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 57.84
Accuracy intervalo: [34.29, 81.39]


In [103]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [72.30, 79.26]


In [104]:
#cv = StratifiedKFold(n_splits = 10, shuffle=True) usar quando tem diferenças entre as classes

from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.55, 78.01]


In [105]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.83, 77.73]


Dados Agrupáveis

In [106]:
np.random.seed(SEED)
np.random.randint(-2, 3, size=10000)

array([-2,  2,  0, ...,  0,  2,  2])

In [107]:
np.random.seed(SEED)
dados["modelo_aleatorio"] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [108]:
dados.modelo_aleatorio.unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1], dtype=int64)

In [109]:
dados.modelo_aleatorio.min()

-1

In [110]:
abs(dados.modelo_aleatorio.min())

1

In [111]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [112]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo_aleatorio.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio,modelo
0,30941.02,1,18,35085.22134,18,18
1,40557.96,1,20,12622.05362,24,24
2,89627.50,0,12,11440.79806,14,14
3,95276.14,0,3,43167.32682,6,6
4,117384.68,1,4,12770.11290,5,5


In [113]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1], dtype=int64)

In [114]:
dados.modelo.value_counts()

modelo
20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: count, dtype: int64

In [115]:
# Não pode usar por não ser realista 

# from sklearn.model_selection import StratifiedKFold

# SEED = 301
# np.random.seed(SEED)

# cv = StratifiedKFold(n_splits = 10, shuffle=True)
# modelo = DecisionTreeClassifier(max_depth=2)
# results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
#imprime_resultados(results)

In [116]:
# from sklearn.modelo_selection import KFold

# SEED = 301
# np.random.seed(SEED)

# cv = KFold(n_splits = 10)
# modelo = DecisionTreeClassifier(max_depth=2)
# results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
# imprime_resultados(results)

In [117]:
# Faltou os grupos 

# from sklearn.model_selection import GroupKFold

# SEED = 301
# np.random.seed(SEED)

# cv = GorupKFold(n_splits = 10)
# modelo = DecisionTreeClassifier(max_depth=2)
# results = cross_validate(modelo, x_azar, cv = cv, return_train_score=False)
# imprime_resultados(results)

In [118]:
import numpy as np
from sklearn.model_selection import GroupKFold, cross_validate
from sklearn.tree import DecisionTreeClassifier

In [119]:
import numpy as np
from sklearn.model_selection import GroupKFold, cross_validate
from sklearn.tree import DecisionTreeClassifier

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=True)
imprime_resultados(results)



Accuracy médio: 75.78
Accuracy intervalo: [73.67, 77.90]


In [121]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head

<bound method NDFrame.head of           preco  vendido  idade_do_modelo   km_por_ano  modelo_aleatorio  \
0      30941.02        1               18  35085.22134                18   
1      40557.96        1               20  12622.05362                24   
2      89627.50        0               12  11440.79806                14   
3      95276.14        0                3  43167.32682                 6   
4     117384.68        1                4  12770.11290                 5   
...         ...      ...              ...          ...               ...   
9995   97112.86        0               12  25060.64248                14   
9996  107424.63        1               16  21317.31764                17   
9997   93856.99        0                4  20950.38812                 6   
9998   51250.57        1                7  16840.13376                11   
9999   87945.73        0               19  19894.66108                23   

      modelo  
0         18  
1         24  
2         14

In [123]:
#Testando validação crizada com GroupKFold

from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.67, 77.90]


In [124]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi de %.2f %%" % acuracia)

A acurácia foi de 77.48 %


In [125]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
imprime_resultados(results)

Accuracy médio: 77.27
Accuracy intervalo: [74.35, 80.20]


In [127]:
# from sklearn.model_selection import GroupKFold

# SEED = 301
# np.random.seed(SEED)

# cv = GroupKFold(n_splits = 10)
# modelo = SVC()
# results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo,return_train_score=False)
# imprime_resultados(results)

In [128]:
from sklearn.pipeline import Pipeline
scaler =StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])
pipeline

Pipeline(steps=[('transformacao', StandardScaler()), ('estimador', SVC())])

In [129]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler =StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
result = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 77.27
Accuracy intervalo: [74.35, 80.20]
